## Menu Item Point of Sale SQL Database
#### with Referencing Recipes & Subrecipes

First draft of a custom restaurant operations solution replacing the standard POS with an All in one
predictive ordering and self govering inventory for Hotels, Collges, Contract Food Service and Beyond.

Upon menu item query you can recieve all pos information, a full complete portioned recipe with any additional
subrecipes associated with query for use as a kitchen prep sheet. I will be building this out continually in effort 
to realize the dream machine of a predictive ordering system based on live in-house computer vison assited inventory 
mechanism as well as some other seed to service systems.

In [1]:
'''SQLite DB backend setup for SQLalchemy '''

import sqlite3
from sqlite3 import Error
import pandas as pd
print("pySqlite3",sqlite3.version, "Sqlite3 v.",sqlite3.sqlite_version, "Pandas", pd.__version__)

pySqlite3 2.6.0 Sqlite3 v. 3.29.0 Pandas 0.25.1


In [2]:
 # create & test a database connection
database = r"D:\Githubrepos\meal_plan_mvp\Database\meal_plan.sqlite"
conn = sqlite3.connect(database)
c = conn.cursor()
print("Opened database successfully")

Opened database successfully


In [3]:
def create_tables(database, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: sqlite3.connect object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        sqliteConnection = sqlite3.connect(database)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")
        cursor.execute(create_table_sql)
        if cursor.rowcount == 0:
            print(f"FAILED: Table {create_table_sql[:30]} failed. rowcount:", cursor.rowcount)
        else:
            print(f"Successfully created table:{create_table_sql[:30]}")
        
  
    except Error as e:
        print(e)
        
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed")
        
def insert_record(database, sqlite_insert_query):
    """Insert record into tables 
    it is enforcing non duplicates but NEEDS TO RAISE ERROR FOR DUPLICATE RECORD?? IF Possible"""
    try:
        sqliteConnection = sqlite3.connect(database)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")

        count =cursor.execute(sqlite_insert_query)
        sqliteConnection.commit()
        if cursor.rowcount == 0:
            print(f"FAILED: Record {sqlite_insert_query[:30]} inserted failed. rowcount:", cursor.rowcount)
        else:
            print(f"Record {sqlite_insert_query[:30]} inserted successfully.")
                  
        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert data into sqlite table", error)
        
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed")

In [4]:
def query_recipe_id(database, sql_query_menu_recipe_id):
    """Query/Print menuitem table record by recipe_id
    returns: ALL menu item info"""
    try:
        sqliteConnection = sqlite3.connect(database)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite")

        count =cursor.execute(sql_query_menu_recipe_id)
        rows = cursor.fetchall()
 
        for row in rows:
            print(row)
        sqliteConnection.commit()
        if cursor.rowcount == 0:
            print(f"FAILED: Record {sql_query_menu_recipe_id[:30]} query failed. rowcount:", cursor.rowcount)
        else:
            print(f"Record {sql_query_menu_recipe_id[:30]} query successfull.")
    except sqlite3.Error as error:
        print("Failed to insert data into sqlite table", error)
        
    finally:
        if (sqliteConnection):
            sqliteConnection.close()
            print("The SQLite connection is closed")

#### Query Functions 
##### Regurgitates MENU ITEM LOCATION, OUTLET, ALL RECIPE INFO, SUBRECIPE

In [5]:
def recipe_full_print(database, recipe_full_print):
    """Query/Print menuitem, recipe, subrecipe tables for full recipe print out
    :param database: sqlite3.connect
    :param recipe_full_print: takes SQL SELECT statement 
    :returns: tuple of menu_item_id, all recipe info plus subrecipe and
        subrecipe of the subrecipe listed in field subrecipe_subrecipe_1,
        and pandas dataframe of fields and values for all of the above
    returns: ALL menu_item_par, recipe, subrecipe info"""
    
    try:
        sqliteConnection = sqlite3.connect(database)
        cursor = sqliteConnection.cursor()
        print("Successfully Connected to SQLite \n") 

        count =cursor.execute(recipe_full_print)
        rows = cursor.fetchall()
        
        for row in rows:
            print(row,"\n")
            
        sqliteConnection.commit()
        
        if cursor.rowcount == 0:
            print(f"FAILED: Record {recipe_full_print[:30]} query failed. rowcount: \n", cursor.rowcount)
        else:
            print(f"Record {recipe_full_print[:30]} query successfull. \n")
        
        #put in a df to cheat and check if second select statment is needed
        recipe_df = pd.read_sql_query(recipe_full_print, sqliteConnection)
        recipe_df = recipe_df.T
        print("Menu Item Recipe in memory ...\n \n")
        
        variable = recipe_df.loc['subrecipe_subrecipe_1'][0]
      
        if recipe_df.loc['subrecipe_subrecipe_1'].any() != "NA":
            print("Adding Second Subrecipe.... \n \n")
            
            try:    
                count =cursor.execute("""SELECT * FROM subrecipe WHERE subrecipe_recipe_name=?""",(variable,))
                print("Executed SELECT subrecipe deets ....")
                rows = cursor.fetchall()
                for row in rows:
                    print(row,"\n")
                
                        
            except sqlite3.Error as error:
                print("Failed to Execute SELECT FROM subrecipe \n", error)
                
            if cursor.rowcount == 0:
                print(f"FAILED: Record subrecipe query failed. rowcount: \n", cursor.rowcount)
            else:
                print(f"Record subrecipe SQL Query successfull. \n")
                print("Subrecipe Data:")
            
            sub_recipe_df = pd.read_sql_query("""SELECT * FROM subrecipe WHERE subrecipe_recipe_name=?""",sqliteConnection, params=(variable,))
            sub_recipe_df = sub_recipe_df.T
            
            print(sub_recipe_df, "\n")
            
            recipe_df = recipe_df.append(sub_recipe_df)
                
            sqliteConnection.commit()
        
            
    except sqlite3.Error as error:
        print("Failed to insert data into sqlite table \n", error)

    if (sqliteConnection):
        sqliteConnection.close()
        print("The SQLite connection is closed \n")
        
        return recipe_df

In [6]:
menu_item_table = """ CREATE TABLE  menuitem (
                                        menu_item_id INTEGER PRIMARY KEY AUTOINCREMENT UNIQUE NOT NULL,
                                        menu_item_created_on INTEGER UNIQUE NOT NULL,
                                        menu_item_location_id VARCHAR NOT NULL,
                                        menu_item_outlet_id VARCHAR NOT NULL,
                                        menu_item_recipe_id TEXT UNIQUE NOT NULL,
                                        menu_item_description TEXT UNIQUE NOT NULL,
                                        menu_item_subscript TEXT NOT NULL,
                                        menu_item_station VARCHAR(50) NOT NULL,
                                        menu_item_pos_name VARCHAR(25) NOT NULL,
                                        menu_item_fire_time INTEGER NOT NULL,
                                        menu_item_par INTEGER NOT NULL,
                                        menu_item_dietary TEXT,
                                        menu_item_menu_price REAL NOT NULL,
                                        menu_item_active_item BIT);"""
                                        
id_create_date_recipe_desc_index = """CREATE UNIQUE INDEX id_create_date_recipe_desc ON menuitem (menu_item_id, menu_item_created_on,
                                                                                 menu_item_recipe_id, menu_item_description);"""
 
recipes_table = """CREATE TABLE  recipes (
                                    recipes_recipe_name TEXT PRIMARY KEY UNIQUE NOT NULL,
                                    recipes_ingredient_1 TEXT NOT NULL,
                                    recipes_ingredient_prep_1 TEXT NOT NULL,
                                    recipes_rcp_quantity_1 REAL NOT NULL,
                                    recipes_measure_1 TEXT NOT NULL,
                                    recipes_ingredient_2 TEXT NOT NULL,
                                    recipes_ingredient_prep_2 TEXT NOT NULL,
                                    recipes_rcp_quantity_2 REAL NOT NULL,
                                    recipes_measure_2 TEXT NOT NULL,
                                    recipes_ingredient_3 TEXT NOT NULL,
                                    recipes_ingredient_prep_3 TEXT NOT NULL,
                                    recipes_rcp_quantity_3 REAL NOT NULL,
                                    recipes_measure_3 TEXT NOT NULL,
                                    recipes_sub_recipe VARCHAR NOT NULL,
                                    recipes_sub_recipe_qty REAL NOT NULL,
                                    recipe_sub_recipe_measure TEXT NOT NULL,
                                    recipes_dietary TEXT,
                                    recipes_prep_instructions VARCHAR NOT NULL,
                                    recipes_fire_instructions VARCHAR NOT NULL,
                                    recipes_est_prep_time REAL NOT NULL,
                                    FOREIGN KEY (recipes_recipe_name) REFERENCES menuitem (menu_item_recipe_id)
                                );"""

id_name_recipe_index = """CREATE UNIQUE INDEX id_recipe_index ON recipes (recipes_recipe_name);"""

subrecipe_table = """CREATE TABLE   subrecipe (
                                    subrecipe_recipe_name TEXT UNIQUE NOT NULL,
                                    subrecipe_ingredient_1 TEXT NOT NULL,
                                    subrecipe_rcp_quantity_1 REAL NOT NULL,
                                    subrecipe_ing_measure_1 TEXT NOT NULL,
                                    subrecipe_ingredient_prep_1 TEXT NOT NULL,
                                    subrecipe_ingredient_2 TEXT NOT NULL,
                                    subrecipe_rcp_quantity_2 REAL NOT NULL,
                                    subrecipe_ingredient_prep_2 TEXT NOT NULL,
                                    subrecipe_ing_measure_2 TEXT NOT NULL,
                                    subrecipe_ingredient_3 TEXT NOT NULL,
                                    subrecipe_rcp_quantity_3 REAL NOT NULL,
                                    subrecipe_ingredient_prep_3 TEXT NOT NULL,
                                    subrecipe_ing_measure_3 TEXT NOT NULL,
                                    subrecipe_subrecipe_1 TEXT,
                                    subrecipe_subrecipe_qty_1 REAL,
                                    subrecipe_subrecipe_measure_1 TEXT,
                                    subrecipe_prep_instructions VARCHAR,
                                    subrecipe_dietary TEXT,
                                    FOREIGN KEY (subrecipe_recipe_name) REFERENCES recipes (recipes_sub_recipe)
                                );"""

subrecipe_name_index = """CREATE UNIQUE INDEX subrecipe_name_index ON subrecipe (subrecipe_recipe_name);"""

 


### CREATE MENUITEM TABLE FOR POS INFO

In [7]:
# create tables
try:
    # create menu_item_table
    create_tables(database, menu_item_table)
    create_tables(database, id_create_date_recipe_desc_index)
        
    #verify tables made
    print("Created table and indexes")

except Error as e:
    print(e)

Successfully Connected to SQLite
Successfully created table: CREATE TABLE  menuitem (
    
The SQLite connection is closed
Successfully Connected to SQLite
Successfully created table:CREATE UNIQUE INDEX id_create_
The SQLite connection is closed
Created table and indexes


In [8]:


sqlite_menu_insert_query = """INSERT OR IGNORE INTO `menuitem`
                          ('menu_item_created_on',
                          'menu_item_location_id',
                          'menu_item_outlet_id',
                          'menu_item_recipe_id',
                          'menu_item_description',
                          'menu_item_subscript',
                          'menu_item_station',
                          'menu_item_pos_name',
                          'menu_item_fire_time',
                          'menu_item_par',
                          'menu_item_dietary',
                          'menu_item_menu_price',
                          'menu_item_active_item') 
                           VALUES 
                          (1570006802,'graduate_seattle','mountaineer','Grilled Chicken','Grilled Free Range Chicken',
                          'mash potato and carrot', 'Grill', 'Grill','12','35','meat, soy, dairy','15.45',1)"""

#format and insert first record on recipes
sqlite_recipe_insert_query = """INSERT OR IGNORE INTO 'recipes'
                          ('recipes_recipe_name',
                          'recipes_ingredient_1',
                          'recipes_ingredient_prep_1',
                          'recipes_rcp_quantity_1',
                          'recipes_measure_1',
                          'recipes_ingredient_2',
                          'recipes_ingredient_prep_2',
                          'recipes_rcp_quantity_2',
                          'recipes_measure_2',
                          'recipes_ingredient_3',
                          'recipes_ingredient_prep_3',
                          'recipes_rcp_quantity_3',
                          'recipes_measure_3',
                          'recipes_sub_recipe',
                          'recipes_sub_recipe_qty',
                          'recipe_sub_recipe_measure',
                          'recipes_dietary', 
                          'recipes_prep_instructions',
                          'recipes_fire_instructions',
                          'recipes_est_prep_time') 
                           VALUES 
                          ('Grilled Chicken','chix breast','whole','8','lbs', 'red bliss potato','quartered','4','oz',
                          'carrots', 'sliced', '3', 'oz.wt.','chausuer sauce', '3', 'oz','meat, soy, dairy',
                          '1. wash chicken 2. braise in red wine and stock 4hrs 3. pull sblah blah blah 4. cryovac & chill for service',
                          '1. Grill 4 mins on each side 2. Saute potatos and peppers 3. Spoon chausuer on plate, scoop mash, rest octo leg over, garnish with carrots',
                          '85')"""

#format and insert first record on subrecipes
sqlite_subrcp_insert = """INSERT OR IGNORE INTO 'subrecipe'
                                    ('subrecipe_recipe_name',
                                    'subrecipe_ingredient_1',
                                    'subrecipe_ingredient_prep_1',
                                    'subrecipe_rcp_quantity_1',
                                    'subrecipe_ing_measure_1',
                                    'subrecipe_ingredient_2',
                                    'subrecipe_ingredient_prep_2',
                                    'subrecipe_rcp_quantity_2',
                                    'subrecipe_ing_measure_2',
                                    'subrecipe_ingredient_3',
                                    'subrecipe_ingredient_prep_3',
                                    'subrecipe_rcp_quantity_3',
                                    'subrecipe_ing_measure_3',
                                    'subrecipe_subrecipe_1',
                                    'subrecipe_subrecipe_qty_1',
                                    'subrecipe_subrecipe_measure_1',
                                    'subrecipe_prep_instructions',
                                    'subrecipe_dietary')
                                    VALUES
                                    ('chausuer sauce','button mushrooms','sliced', '1', 'lb',
                                    'shallot', 'minced','8','oz', 'white wine','pour', '6','oz','demi glace (basic)','1','G',
                                    '1. Saute mushroom & shallot 2.deglaze with white wine 3. Add demi glace and reduce by 1/2',
                                    'meat, gluten'
                                    )"""

##format and insert first record on subrecipe where subrecipe has subrecipe
sqlite_subrcp2_insert = """INSERT OR IGNORE INTO 'subrecipe'
                                    ('subrecipe_recipe_name',
                                    'subrecipe_ingredient_1',
                                    'subrecipe_ingredient_prep_1',
                                    'subrecipe_rcp_quantity_1',
                                    'subrecipe_ing_measure_1',
                                    'subrecipe_ingredient_2',
                                    'subrecipe_ingredient_prep_2',
                                    'subrecipe_rcp_quantity_2',
                                    'subrecipe_ing_measure_2',
                                    'subrecipe_ingredient_3',
                                    'subrecipe_ingredient_prep_3',
                                    'subrecipe_rcp_quantity_3',
                                    'subrecipe_ing_measure_3',
                                    'subrecipe_subrecipe_1',
                                    'subrecipe_subrecipe_qty_1',
                                    'subrecipe_subrecipe_measure_1',
                                    'subrecipe_prep_instructions',
                                    'subrecipe_dietary')
                                    VALUES
                                    ('demi glace (basic)','beef bones','whole', '10', 'lb',
                                    'water', 'cold','12','G', 'garlic','minced','6','oz','NA','NA','NA',
                                    '1. Rinse Bones 2. Cover with water 3. simmer for 8 hours add garlic 4. After 10 hours strain and cool 5. Bag and cryo',
                                    'meat, gluten'
                                    )"""



In [9]:
#insert first menuitem record in order to make recipe table with foreign key           
insert_record(database, sqlite_menu_insert_query)

Successfully Connected to SQLite
Record INSERT OR IGNORE INTO `menuite inserted successfully.
The SQLite connection is closed


In [10]:
# create recipe tables
try:
 
    # create recipes_table after menuitem has foreign key match
    create_tables(database, recipes_table)
    
    #bind unique index to stop duplicate record
    create_tables(database, id_name_recipe_index)
    
    #insert first record
    insert_record(database, sqlite_recipe_insert_query)
        
    #verify tables made
    print("Created table and indexes")

except Error as e:
    print(e)


Successfully Connected to SQLite
Successfully created table:CREATE TABLE  recipes (
      
The SQLite connection is closed
Successfully Connected to SQLite
Successfully created table:CREATE UNIQUE INDEX id_recipe_
The SQLite connection is closed
Successfully Connected to SQLite
Record INSERT OR IGNORE INTO 'recipes inserted successfully.
The SQLite connection is closed
Created table and indexes


## ADD SUBRECIPE TABLE AND INSERT INITIAL RECORD

In [11]:
# create subrecipe tables
try:
 
    # create recipes_table after menuitem has foreign key match
    create_tables(database, subrecipe_table)
    
    #bind unique index to stop duplicate record
    create_tables(database, subrecipe_name_index)
    
    #insert first record
    insert_record(database, sqlite_subrcp_insert)
    
    #second record subrecipe
    insert_record(database, sqlite_subrcp2_insert)
        
    #verify tables made
    print("Created table and indexes")

except Error as e:
    print(e)

Successfully Connected to SQLite
Successfully created table:CREATE TABLE   subrecipe (
   
The SQLite connection is closed
Successfully Connected to SQLite
Successfully created table:CREATE UNIQUE INDEX subrecipe_
The SQLite connection is closed
Successfully Connected to SQLite
Record INSERT OR IGNORE INTO 'subreci inserted successfully.
The SQLite connection is closed
Successfully Connected to SQLite
Record INSERT OR IGNORE INTO 'subreci inserted successfully.
The SQLite connection is closed
Created table and indexes


## DUPLICATE RECORDS CHECK
these should fail to demonstrate the unique index constraints

In [12]:
#test fail state of record duplicate insert subrecipe
insert_record(database, sqlite_recipe_insert_query)

Successfully Connected to SQLite
FAILED: Record INSERT OR IGNORE INTO 'recipes inserted failed. rowcount: 0
The SQLite connection is closed


In [13]:
# double check duplicate record insert sub recipe
insert_record(database, sqlite_subrcp2_insert)

Successfully Connected to SQLite
FAILED: Record INSERT OR IGNORE INTO 'subreci inserted failed. rowcount: 0
The SQLite connection is closed


In [14]:
# double check duplicate record insert menuitem 
insert_record(database, sqlite_menu_insert_query)

Successfully Connected to SQLite
FAILED: Record INSERT OR IGNORE INTO `menuite inserted failed. rowcount: 0
The SQLite connection is closed


## QUERY TEST MENUITEM ID

In [15]:
#sample query by recipe id
sql_query_menu_recipe_id = """SELECT * FROM menuitem WHERE menu_item_recipe_id = 'Grilled Chicken'; """

query_recipe_id(database, sql_query_menu_recipe_id)

Successfully Connected to SQLite
(1, 1570006802, 'graduate_seattle', 'mountaineer', 'Grilled Chicken', 'Grilled Free Range Chicken', 'mash potato and carrot', 'Grill', 'Grill', 12, 35, 'meat, soy, dairy', 15.45, 1)
Record SELECT * FROM menuitem WHERE m query successfull.
The SQLite connection is closed


## TRYING TO CATCH ALL SUB RECIPES CASCADING FROM MENU ITEM --> RECIPES --> SUBRECIPE where SUBRECIPE has SUBRECIPE

In [16]:
recipe_full_print_test = """SELECT menu_item_location_id, menu_item_outlet_id, recipes.*,  subrecipe.*
                            FROM recipes
                                INNER JOIN menuitem ON menuitem.menu_item_recipe_id = recipes.recipes_recipe_name
                                INNER JOIN subrecipe ON subrecipe.subrecipe_recipe_name = recipes.recipes_sub_recipe
                            WHERE menu_item_recipe_id = 'Grilled Chicken'"""
                            
#                            IF subrecipe.subrecipe_subrecipe_1 IS NOT NULL THEN
#                                SELECT subrecipe.subrecipe_name = subrecipe.subrecipe_1 FROM subrecipe;"""


In [17]:
df = recipe_full_print(database, recipe_full_print_test)

Successfully Connected to SQLite 

('graduate_seattle', 'mountaineer', 'Grilled Chicken', 'chix breast', 'whole', 8.0, 'lbs', 'red bliss potato', 'quartered', 4.0, 'oz', 'carrots', 'sliced', 3.0, 'oz.wt.', 'chausuer sauce', 3.0, 'oz', 'meat, soy, dairy', '1. wash chicken 2. braise in red wine and stock 4hrs 3. pull sblah blah blah 4. cryovac & chill for service', '1. Grill 4 mins on each side 2. Saute potatos and peppers 3. Spoon chausuer on plate, scoop mash, rest octo leg over, garnish with carrots', 85.0, 'chausuer sauce', 'button mushrooms', 1.0, 'lb', 'sliced', 'shallot', 8.0, 'minced', 'oz', 'white wine', 6.0, 'pour', 'oz', 'demi glace (basic)', 1.0, 'G', '1. Saute mushroom & shallot 2.deglaze with white wine 3. Add demi glace and reduce by 1/2', 'meat, gluten') 

Record SELECT menu_item_location_id,  query successfull. 

Menu Item Recipe in memory ...
 

Adding Second Subrecipe.... 
 

Executed SELECT subrecipe deets ....
('demi glace (basic)', 'beef bones', 10.0, 'lb', 'whole',

In [18]:
df.reset_index()
df.rename(columns={'index': 'FIELD', '0':'PREP_LIST'})

,0
menu_item_location_id,graduate_seattle
menu_item_outlet_id,mountaineer
recipes_recipe_name,Grilled Chicken
recipes_ingredient_1,chix breast
recipes_ingredient_prep_1,whole
recipes_rcp_quantity_1,8
recipes_measure_1,lbs
recipes_ingredient_2,red bliss potato
recipes_ingredient_prep_2,quartered
recipes_rcp_quantity_2,4


## NEED TO FURTHER BUILD OUT:

- BEV & STOCKED INV ITEMS / ANCILLARY

- ADD MENU ITEM, RECIPE, SUB FUNCTIONS TO CATCH FRONT END HOOKS

- "PULL FROM" STORAGE AREA COLUMN

- "GROUP BY" STORAGE AREA & RECIPES PER STATION VIEW

- ABILITY TO ADD INGREDIENTS LINES FOR RECIPES AND SUBRECIPES THAT BRINGS IN PREP QTY AND MEASURE FROM HOOKS

- TEST RECIPES WITH 2 + SUB RECIPES

- FORMAT RECIPE LOOK WITH ROOM FOR PREP INSTRUCTIONS AT THE BOTTOM OF EACH

- AUTO POPULATE MENUITEM DB, RECIPE AND SUBS WITH ALLERGEN INFO FROM VENDOR INFO

- LOG REGRESSION, RNN & DL MODEL WITH RANDOM SAMPLE SALES DATA IN FULL CYCLE
